## Day 1

In [2]:
with open('data/input01.txt') as f:
    l = f.readlines()[0].strip("\n")

print("Part 1:",sum([ int(a) for a,b in zip(l[0:],l[1:]+l[0]) if a==b]))
print("Part 2:",sum([ int(a) for a,b in zip(l[0:],l[len(l)//2:]+l[0:len(l)//2]) if a==b]))

Part 1: 1223
Part w: 1284


## Day 2

In [29]:
with open('data/input02.txt') as f:
    spreadsheet = [ [ int(i) for i in l.strip("\n").split("\t") ] for l in f.readlines() ]

checksum = 0
for s in spreadsheet:
    checksum += max(s)-min(s)
print("Part 1:",checksum)

from itertools import permutations

checksum = 0
for s in spreadsheet:
    for i,j in permutations(s,2):
        if i!=j and i//j == i/j:
            checksum += i//j
print("Part 2:",checksum)  

Part 1: 45972
Part 2: 326


## Day 3

In [61]:
def distance(target):
    d = 1
    v = 1
    inc = 1
    p = 0 
    while True:
        for side in (1,2):
            for _ in range(inc):
                v+=1 # increment value
                p+=d # change position in spiral
                if v==target:
                    return int(abs(p.real)+abs(p.imag)) 
            d *= 1j # change directions using complex numbers
        inc +=1
        
print("Part 1:",distance(361527))

from collections import defaultdict

def neighbours(p):
    return [ p-1+1j , p+1j , p+1+1j,
             p-1    ,        p+1   , 
             p-1-1j , p-1j , p+1-1j ]

def spiral(target):
    d = 1
    v = 1
    inc = 1
    p = 0 
    value = defaultdict(lambda: 0) # store values according to (complex) position
    value[p] = v
    while True:
        for side in (1,2):
            for _ in range(inc):
                p+=d # change position in spiral
                v = 0
                for n in neighbours(p): # sum values of neigbours, if present (0 otherwise)
                    v += value[n] 
                if v>target:
                    return v
                value[p] = v
            d *= 1j # change directions using complex numbers
        inc +=1
        
print("Part 2:",spiral(361527))

Part 1: 326
Part 2: 363010


## Day 4

In [86]:
with open("data/input04.txt") as f:
    phrases = [ [ w for w in p.strip("\n").split(" ") ] for p in f.readlines() ]

from collections import Counter

valid = 0
for p in phrases:
    count = Counter(p)
    if count.most_common()[0][1] == 1:
        valid += 1
print("Part 1:",valid)

from itertools import permutations

notvalid = 0
for p in phrases:
    for a,b in permutations(p,2):
        if sorted(a)==sorted(b):
            notvalid +=1
            break
print("Part 2:",len(phrases)-notvalid)

Part 1: 477
Part 2: 167


## Day 5

In [144]:
def steps(instr,part=1,verbose=False):
    offset = 0
    steps = 0
    i = 0
    while i<len(instr):
        jump = instr[i]
        if verbose: print(i,instr,jump)
        if part==2 and jump >=3:
            instr[i] -= 1
        else:
            instr[i] += 1
        i += jump
        steps +=1
    return(steps)

instr_test = [0,3,0,1,-3 ]

with open("data/input05.txt") as f:
    instr = [ int(i) for i in f.readlines() ]

# don't forget to copy the instructions with list()!
print("Test 1:",steps(list(instr_test)))
print("Part 1:",steps(list(instr))) 
print("Test 2:",steps(list(instr_test),part=2))
print("Part 2:",steps(list(instr),part=2))

Test 1: 5
Part 1: 359348
Test 2: 10
Part 2: 27688760


## Day 6

In [197]:
def redistribute(bank):
    m = max(bank)
    im = bank.index(m)
    bank[im]=0
    j = (im+1)%len(bank)
    for _ in range(m):
        bank[j] += 1
        j = (j+1)%len(bank)
    return bank

from collections import defaultdict

def cycles(bank,verbose=False):
    if verbose: print(bank)
    states = defaultdict(lambda:0)
    cycles = 0
    states["-".join([str(i) for i in bank])]=cycles
    while True:
        bank = redistribute(bank)
        cycles +=1
        if verbose: print(bank)
        h = "-".join([str(i) for i in bank])
        if states[h]>0:
            return cycles,states[h]
            break
        else:
            states[h]=cycles

cycl, first = cycles([0, 2, 7, 0],True)
print("Test 1:",cycl)
print("Test 2:",cycl-first)

with open("data/input06.txt") as f:
    bank = [ int(i) for i in f.readlines()[0].strip("\n").split("\t")]

cycl, first = cycles(bank)
print("Part 1:",cycl)
print("Part 2:",cycl-first)

[0, 2, 7, 0]
[2, 4, 1, 2]
[3, 1, 2, 3]
[0, 2, 3, 4]
[1, 3, 4, 1]
[2, 4, 1, 2]
Test 1: 5
Test 2: 4
Part 1: 5042
Part 2: 1086


## Day 7

In [159]:
def readInput7(filename):
    weight = {}
    node = {}
    with open(filename) as f:
        for l in f.readlines():
            a = l.strip("\n").split(" -> ")
            weight[a[0].split(" ")[0]] = int(a[0].split(" ")[1].replace("(","").replace(")",""))
            if len(a)==2:
                node[a[0].split(" ")[0]] = a[1].split(", ")
    return weight,node

def findBottom(node,weight):
    s = list(weight.keys())[0]    
    while True:
        i = 0
        for n in node.keys():
            if s in node[n]:
                s = n
                break
            i += 1
        if i==len(node):
            return s

weight,node = readInput7("data/day07example.txt")
print("Test 1:",findBottom(node,weight))

weight,node = readInput7("data/input07.txt")
print("Part 1:",findBottom(node,weight))

def towerWeight(n,node,weight):
    if n in node.keys():
        return weight[n] + sum( [ towerWeight(m,node,weight) for m in node[n] ] )
    else:
        return weight[n]

from collections import Counter

def findDelta(node,weight):
    # find faulty weight
    b = findBottom(node,weight)
    while True:
        counter = Counter([towerWeight(w,node,weight) for w in node[b]])
        if len(counter)==1: # change is in node
            break
        else: # change is in leaves        
            for n in node[b]:
                if towerWeight(n,node,weight) == counter.most_common()[1][0]:
                    b = n
                    break
            if b in node.keys():
                pass
            else:
                break
    # fix faulty weight
    for n in node.keys():
        if b in node[n]:
            counter = Counter([towerWeight(w,node,weight) for w in node[n]])
            break
    return b,counter.most_common()[0][0]-counter.most_common()[1][0]

weight,node = readInput7("data/day07example.txt")
b,delta = findDelta(node,weight)
print("Test 2:",b,"\t delta =", delta,", weight =",weight[b],"->",weight[b]+delta)

weight,node = readInput7("data/input07.txt")
b,delta = findDelta(node,weight)
print("Part 2:",b,"\t delta =", delta,", weight =",weight[b],"->",weight[b]+delta)

Test 1: tknk
Part 1: bpvhwhh
Test 2: ugml 	 delta = -8 , weight = 68 -> 60
Part 2: tulwp 	 delta = -8 , weight = 264 -> 256
